In [1]:
import panel as pn
import ipyvuetify as v
import traitlets
import ipywidgets as widgets

pn.extension()

In [2]:
items = [
      { 'text': 'First', 'icon': 'mdi-clock', 'time':  10},
      { 'text': 'Second', 'icon': 'mdi-account', 'time': 20 },
      { 'text': 'Third', 'icon': 'mdi-flag', 'time': 30 },
    ]

class SimpleList(v.VuetifyTemplate):
    selectedItem = traitlets.Int(allow_none=True).tag(sync=True)
    items = traitlets.List(items).tag(sync=True)
    
    template = traitlets.Unicode('''
    <v-card
      class="mx-auto"
      max-width="300"
      tile
    >
      <v-list flat>
        <v-subheader>TIMESTAMPS</v-subheader>
        <v-list-item-group
          v-model="selectedItem"
          color="primary"
        >
          <v-list-item
            v-for="(item, i) in items"
            :key="i"
          >
            <v-list-item-icon>
              <v-icon v-text="item.icon"></v-icon>
            </v-list-item-icon>
            <v-list-item-content>
              <v-list-item-title v-text="item.text"></v-list-item-title>
              {{item.time}} seconds
            </v-list-item-content>
          </v-list-item>
        </v-list-item-group>
      </v-list>
    </v-card>
    ''').tag(sync=True)
    
simple_list = SimpleList()

In [3]:
class MyApp(widgets.VBox):
    VIDEO_URL = 'https://file-examples-com.github.io/uploads/2017/04/file_example_MP4_640_3MG.mp4'
    def __init__(self):
        super().__init__()
        self.video = pn.pane.Video(self.VIDEO_URL, width=640, height=360, loop=False)
        self.ipy_video = pn.ipywidget(self.video)
        self.my_list  = SimpleList()
        self.out = widgets.Output()
        
        self.video.param.watch(self.update_ts, 'time')
        self.my_list.observe(self.on_list_change, names=['selectedItem'])
        
        self.children = [widgets.HBox(children=[self.ipy_video, self.my_list]), self.out]
        
        
    def on_list_change(self, change):
        print(change)
        if change['new'] is not None:
            items =  change['owner'].items
            item_idx = change['new']
            item = items[item_idx]
            self.video.time = item['time']

    def update_ts(self, event):
        self.out.clear_output()
        with self.out:
            print('new ts: ', event)
            
app = MyApp()
app

MyApp(children=(HBox(children=(BokehModel(combine_events=True, render_bundle={'docs_json': {'e6b3a4a1-6210-4a6…

When selecting an entry in the list the time in the video is updated both in Jupyter Lab and Voila.
But, when changing the time in the video player, the event is only updating the `time` property of the video in Jupyter Lab but not in Voila.